In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import os 
import pandas as pd
import numpy as np 
import transformers
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
import torchmetrics
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger
from transformers import MobileBertModel, MobileBertConfig, MobileBertTokenizer, MobileBertForSequenceClassification


In [3]:
full_data = pd.read_csv('../input/feedback-prize-english-language-learning/train.csv')

In [4]:
full_data.head(n=20)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5
5,004AC288D833,"Dear Principal,\r\n\r\nOur school should have ...",3.5,4.0,4.0,3.5,3.5,4.0
6,005661280443,Imagine if you could prove other people that y...,3.5,4.0,3.5,3.5,4.0,4.0
7,008DDDDD8E8D,I think it's a good idea for the estudnets to ...,2.5,2.5,2.5,2.5,2.5,2.0
8,009BCCC61C2A,positive attitude is the key to success. I agr...,3.0,3.0,3.5,3.5,3.0,3.0
9,009F4E9310CB,Asking more than one person for and advice hel...,3.0,3.0,3.5,2.5,3.0,2.5


In [5]:
train_df, val_df = train_test_split(full_data, test_size = 0.05)

In [6]:
train_df.shape, val_df.shape

((3715, 8), (196, 8))

In [7]:
Label_Columns = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

In [8]:
class TestAESDataset(pl.LightningDataModule):
    
    def __init__(
        self,
        data,
        tokenizer,
        max_token_len = 512
        ):
        self.tokenizer - tokenizer
        self.data = data
        self.max_token_len = max_token_len
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index: int):
        data_row= self.data.iloc[index]
        comment_text

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [10]:
train_csv = pd.read_csv("../input/feedback-prize-english-language-learning/train.csv")

In [11]:
train_csv['labels']=''

In [12]:
for index, row in train_csv.iterrows():
    train_csv.at[index, 'labels'] = np.zeros((6))
    train_csv.at[index, 'labels'][0] = row['cohesion']
    train_csv.at[index, 'labels'][1] = row['syntax']
    train_csv.at[index, 'labels'][2] = row['vocabulary']
    train_csv.at[index, 'labels'][3] = row['phraseology']
    train_csv.at[index, 'labels'][4] = row['grammar']
    train_csv.at[index, 'labels'][5] = row['conventions']
   

In [13]:
train_csv 

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,labels
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,"[3.5, 3.5, 3.0, 3.0, 4.0, 3.0]"
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]"
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,"[3.0, 3.5, 3.0, 3.0, 3.0, 2.5]"
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,"[4.5, 4.5, 4.5, 4.5, 4.0, 5.0]"
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,"[2.5, 3.0, 3.0, 3.0, 2.5, 2.5]"
...,...,...,...,...,...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,3.0,3.0,3.5,2.5,2.5,"[2.5, 3.0, 3.0, 3.5, 2.5, 2.5]"
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,4.0,4.0,4.0,3.5,3.0,"[4.0, 4.0, 4.0, 4.0, 3.5, 3.0]"
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,3.0,3.0,3.0,3.5,3.0,"[2.5, 3.0, 3.0, 3.0, 3.5, 3.0]"
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,4.5,4.5,4.0,4.5,4.5,"[4.0, 4.5, 4.5, 4.0, 4.5, 4.5]"


In [14]:
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/847 [00:00<?, ?B/s]

In [15]:

def tokenizer_function(text):
    return tokenizer(text['full_text'], add_special_tokens=True, max_length=512,truncation=True, padding='max_length')

In [16]:
train_full = Dataset.from_dict(train_csv)

In [17]:
train_full

Dataset({
    features: ['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions', 'labels'],
    num_rows: 3911
})

In [18]:
train_tokenized = train_full.map(tokenizer_function, batched= True)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [19]:
train_tokenized.column_names

['text_id',
 'full_text',
 'cohesion',
 'syntax',
 'vocabulary',
 'phraseology',
 'grammar',
 'conventions',
 'labels',
 'input_ids',
 'token_type_ids',
 'attention_mask']

In [20]:
train_tokenized = train_tokenized.remove_columns(['full_text', 'text_id', 'cohesion',
 'syntax',
 'vocabulary',
 'phraseology',
 'grammar',
 'conventions'])

In [21]:
train_tokenized.column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [22]:
train_tokenized

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3911
})

In [23]:
train_tokenized = train_tokenized.train_test_split(train_size=0.95)

In [24]:
train_tokenized

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3715
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 196
    })
})

In [25]:
type(train_tokenized['train']['labels'][0])

list

In [26]:
model_config =  MobileBertConfig.from_pretrained('google/mobilebert-uncased')


In [27]:
class AESDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = '../input/feedback-prize-english-language-learning/', batch_size: int = 64, max_length: int = 512):
        super().__init__()
        self.tokenizer= MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.max_length= max_length
    
    def tokenizer_function(self, text):
        return self.tokenizer(text['full_text'], max_length=512,truncation=True, padding='max_length')
  
    def setup(self, stage):
        train_csv = pd.read_csv("../input/feedback-prize-english-language-learning/train.csv")
        train_csv['labels']=''
        for index, row in train_csv.iterrows():
            train_csv.at[index, 'labels'] = np.zeros((6))
            train_csv.at[index, 'labels'][0] = row['cohesion']
            train_csv.at[index, 'labels'][1] = row['syntax']
            train_csv.at[index, 'labels'][2] = row['vocabulary']
            train_csv.at[index, 'labels'][3] = row['phraseology']
            train_csv.at[index, 'labels'][4] = row['grammar']
            train_csv.at[index, 'labels'][5] = row['conventions']    
        train_full = Dataset.from_dict(train_csv)
        test_full = load_dataset("csv", data_files= '../input/feedback-prize-english-language-learning/test.csv')
        train_tokenized = train_full.map(self.tokenizer_function, batched=True)
        test_tokenized = test_full.map(self.tokenizer_function, batched=True)
        train_tokenized = train_tokenized.remove_columns(['full_text', 'text_id', 'cohesion','syntax','vocabulary','phraseology','grammar','conventions'])
        test_tokenized = test_tokenized.remove_columns(['full_text', 'text_id'])
        train_tokenized = train_tokenized.train_test_split(train_size=0.95)
        self.train_ds = train_tokenized['train']
        self.val_ds = train_tokenized['test']
        self.test_ds = test_tokenized['train']
   
    def train_dataloader(self):
        return DataLoader(self.train_ds, self.batch_size, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.val_ds, self.batch_size, num_workers=2)
    
    def test_dataloader(self):
        return DataLoader(self.test_ds, self.batch_size, num_workers=2)


In [28]:
BATCH_SIZE = 12
data_module = AESDataModule(batch_size=BATCH_SIZE)
NUM_CLASSES = 6 

In [29]:
class AESModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.loss = nn.MSELoss()
        self.gain = nn.CosineSimilarity()
        #We need a couple of scalars to mix-up the losses
        self.alpha = 1
        self.beta = 0.001
        self.backbone = MobileBertForSequenceClassification.from_pretrained('google/mobilebert-uncased', num_labels=6)
        
    def forward(self,input_ids, attention_mask):
        x = self.backbone(input_ids, attention_mask).logits
        return(x)
    
    def training_step(self,batch,batch_idx):
        input_ids = torch.stack(batch['input_ids'], dim=1)
        
        attention_mask = torch.stack(batch['attention_mask'], dim=1)
    
        target = torch.stack(batch['labels'], dim=1)
        
        outputs = self(input_ids, attention_mask)
        loss = (self.loss(outputs.float(), (target.float() - 3.0)/2) 
                + self.alpha*torch.mean((1-self.gain(outputs.float(), (target.float() - 3.0)/2)))
                + self.beta*(torch.mean(torch.norm(outputs.float(), p='fro', dim=1))))
        
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self,batch,batch_idx):
        input_ids = torch.stack(batch['input_ids'], dim=1)
        attention_mask = torch.stack(batch['attention_mask'], dim=1)
        target = torch.stack(batch['labels'], dim=1)
        
        outputs = self(input_ids, attention_mask)
        loss = self.loss(outputs.float(), (target.float() - 3.0)/2)
        
        self.log("val_loss" ,loss, prog_bar=True )
        return {"loss": loss, "logits": outputs, "labels": target}
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters() , 1e-3)
        sch = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma=0.1)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": sch,
                "monitor": "val_loss",
            },
        }

In [30]:
model= AESModule()

Downloading:   0%|          | 0.00/140M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.dense.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some 

In [31]:
logger = TensorBoardLogger("light_logs/", name="AESlog")
checkpoint_callback = ModelCheckpoint(dirpath="checkpoints/",
    filename="best-checkpoint", save_top_k=1, verbose=True, 
    monitor="val_loss", mode="min")

In [32]:
trainer = pl.Trainer(max_epochs=50, gpus=1, callbacks=[checkpoint_callback])

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"


In [ ]:
trainer.fit(model, data_module)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c6b57e8e3ec88319/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /kaggle/working/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]